In [ ]:
#Task 4

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.tools.plotting import scatter_matrix
from sklearn.decomposition import PCA
from sklearn import decomposition
#loading data
Mydata=pd.read_excel('chronic_kidney_disease_full.xls')

#interpolating missing values
Mydata=Mydata.interpolate(method="linear")
Mydata=Mydata.fillna(method='pad')
Mydata=Mydata.fillna(method='backfill')


In [ ]:
#Part a:Performing Principal component analysis 

#determining classes
list_Classes=['notckd','ckd'] 
Mydata=Mydata.loc[Mydata['class'].isin(list_Classes)]

#making a copy of actual data set and removing nominal attribues
numericaldata=Mydata
numericaldata=numericaldata.drop(['rbc','pc','pcc','ba','htn','dm','cad','appet','pe','ane','class'],axis=1)
#performing PCA
pca=PCA(n_components=len(numericaldata.columns))
pca_final_result=pca.fit_transform(numericaldata)

#plotting: showing fraction of the overall variance contained in n principal components
list_Var_A=np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)
index_A = np.arange(len(pca.explained_variance_ratio_))
plt.figure(figsize=(9, 3))
plt.title("Fraction of the overall variance in the data")
plt.xlabel("Components numbers")
plt.ylabel("Cumulative Explained Variance")
plt.plot(index_A,list_Var_A, 'bo-')
plt.ylim(99.80, 100)
plt.savefig("C:\\Users\\Tamal\\Documents\\3rd Semester\\Bioinfo ll\OutPuts\\MyPlot.png")

#How many components needed to cover >=99.99% of the variance
component=0
for i in range(len(list_Var_A)):
    if list_Var_A[i]>=99.99:
        component=i
        break
print (str(component+1)+" components needed to cover 99.99% of the variance")

In [ ]:
#Part b: creating a scatter plot matrix with first five principal components
#!pip install seaborn #using seaborn as a visualization library
five_components=pca_final_result[:,:5]
five_comp_df = pd.DataFrame(five_components, columns=['1-PC', '2-PC', '3-PC', '4-PC','5-PC'])
five_comp_df
import seaborn as sns;
sns.set(style="ticks", color_codes=True)
five_comp_df['class']=list(Mydata['class'])
g = sns.pairplot(five_comp_df,hue='class') #creating the plot
plt.savefig("C:\\Users\\Tamal\\Documents\\3rd Semester\\Bioinfo ll\OutPuts\\myScatterPlot.png")

In [ ]:
#Part c: Detect outliers
Mydata['ID'] = Mydata.index
def Outliers_detector(five_components):
    ID=[]
    data=five_components[:,2] 
    box=plt.boxplot(data)
    outliers=box["fliers"][0].get_data()[1]
    ID=np.in1d(data.ravel(),outliers)
    return ID

list_Indices=Outliers_detector(five_components)
outlier_IDs=Mydata.loc[list_Indices]['ID']
print ("List of outlier IDs: "+str(outlier_IDs))

In [ ]:
#Part d: normalizing data 
nomalized_data = pd.DataFrame({col: (numericaldata[col] - numericaldata[col].min()) / (numericaldata[col].max() - numericaldata[col].min())
                               for col in numericaldata.columns})
#doing the PCA analysis
nomalized_PCA = decomposition.PCA()
nomalized_PCA.fit(nomalized_data)
#print(nomalized_data)
#creating the plot
plt.title("PCA on Normalized Data")
plt.xlabel("Number Components")
plt.ylabel("Cumulative Explained Variance")
plt.plot(np.cumsum(nomalized_PCA.explained_variance_ratio_), 'bo-')
plt.savefig("C:\\Users\\Tamal\\Documents\\3rd Semester\\Bioinfo ll\OutPuts\\nomalized_PCAPlot.png")